# OCR with Pytesseract and OpenCV

Pytesseract or Python-tesseract is an OCR tool for python that also serves as a wrapper for the Tesseract-OCR Engine. It can read and recognize text in images and is commonly used in python ocr image to text use cases.

in frest we need to install opencv and pytesseract lib in our environment variable by below commands

#### pip install pytesseract

#### pip install opencv-python


then import them in your code


In [2]:
import cv2
import pytesseract
import numpy as np

first step of our work is read image with cv2

In [3]:
img = cv2.imread('data/simple/drug.jpg')

then config tesseract

In [4]:
custom_config = r'--oem 3 --psm 6'
pytesseract.image_to_string(img, config=custom_config)

TypeError: Unsupported image object

# Preprocessing for Tesseract
To avoid all the ways your tesseract output accuracy can drop, you need to make sure the image is appropriately pre-processed.This includes rescaling, binarization, noise removal, deskewing, etc.

###### To preprocess image for OCR, use any of the following python functions

In [22]:
# get grayscale image
def get_grayscale(image):
    return cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# noise removal
def remove_noise(image):
    return cv2.medianBlur(image,5)

#thresholding
def thresholding(image):
    return cv2.threshold(image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]

#dilation
def dilate(image):
    kernel = np.ones((5,5),np.uint8)
    return cv2.dilate(image, kernel, iterations = 1)

#erosion
def erode(image):
    kernel = np.ones((5,5),np.uint8)
    return cv2.erode(image, kernel, iterations = 1)

#opening - erosion followed by dilation
def opening(image):
    kernel = np.ones((5,5),np.uint8)
    return cv2.morphologyEx(image, cv2.MORPH_OPEN, kernel)

#canny edge detection
def canny(image):
    return cv2.Canny(image, 100, 200)

#skew correction
def deskew(image):

    coords = np.column_stack(np.where(image > 0))
    angle = cv2.minAreaRect(coords)[-1]
    if angle < -45:
        angle = -(90 + angle)
    else:
        angle = -angle
    (h, w) = image.shape[:2]
    center = (w // 2, h // 2)
    M = cv2.getRotationMatrix2D(center, angle, 1.0)
    rotated = cv2.warpAffine(image, M, (w, h), flags=cv2.INTER_CUBIC, borderMode=cv2.BORDER_REPLICATE)
    return rotated

#template matching
def match_template(image, template):
    x = cv2.matchTemplate(image, template, cv2.TM_CCOEFF_NORMED)
    return x



After  **preprocessing** with the following code

In [55]:

w =  pytesseract.image_to_string(img, config=custom_config)

info = {}

for index , line in enumerate(w.split('\n')):
    if index == 0 :
        # TODO  and line.lower().find('rx')
        continue
    elif index == 1:
        info['medicine_name'] = line
        continue
    elif index == 2 :
        info['usage'] = line
        continue

    key_val = line.split(':')
    if len(key_val) > 0 and len(key_val[0]):
        info[key_val[0]] = "".join(key_val[1:])



print(info)




{'medicine_name': '. APO-DOXAZOSIN 1 MG TABI', 'usage': '1 Tablet(s) Once daily X 90 Day(s) starting on 20', 'Qty': ' 90 Tablet(s) (Ninety)', 'Refills': ' None', 'Drug Use': ' _Continuous/Chronic', 'Route': ' Oral DIN 02240588'}
